#### Feature Engineering 

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

#### Categorical features

In [2]:
df = pd.read_csv('Churn_Banking_competition.csv', encoding="latin-1")

for col in df.columns:
    if col.startswith('Flag'):
        df[col] = df[col].astype('object')


numeric_variables_count = len(df.select_dtypes(include=['number']).columns)
categorical_variables_count = len(df.select_dtypes(include=['object']).columns)

print("number of numeric features",numeric_variables_count)
print("number of categorical features",categorical_variables_count)

print(df.describe(include=['object']))

number of numeric features 22
number of categorical features 20
        Flag_Apertura_Conto_Online  Flag_Possesso_piu_Conti  \
count                       175331                   175331   
unique                           2                        2   
top                              0                        0   
freq                        134126                   162608   

       Provincia_Domicilio Provincia_Residenza Professione  \
count               165351              166140      158576   
unique                 112                 111          17   
top                     RM                  RM   Impiegato   
freq                 24696               23534       48514   

                   Imp_Reddito Tipologia_Cliente   Sesso Profilo_MIFID  \
count                    72783            175329  175330        152455   
unique                       5                 5       2             5   
top     Alto (tra 1800 e 2500)            Banker       M      DINAMICO   
freq         

###### The features 'Professione' has two same modalities, so we merged them.

In [3]:
df_new=df
df_new['Professione'] = df_new['Professione'].apply(lambda x: 'Ufficiale/Sottoufficiale' if x == 'Ufficiale/Sottufficiale' else 
                                                                  'Libero professionista/Titolare impresa' if x == 'Libero professionista/Titolare di impresa' else 
                                                                  x)
df_new['Professione'].unique()

array(['Impiegato', 'Libero professionista/Titolare impresa',
       'Altro/Nessuno', 'Avvocato', nan, 'Quadro', 'Ingegnere/Architetto',
       'Medico/Farmacista', 'Docente/Insegnante', 'Rappresentante/Agente',
       'Ufficiale/Sottoufficiale', 'Commercialista',
       'Commesso/Ausiliario', 'Operaio', 'Dirigente', 'Notaio',
       'Autotrasportatore'], dtype=object)

###### The features Provincia_Residenza and Provincia_Domicilio have different modalities referred to the same place, for example, for Forlì-Cesena exist FO and FC, so we merged them. 

In [4]:
domicilio_counts = df_new['Provincia_Domicilio'].value_counts()
residenza_counts = df_new['Provincia_Residenza'].value_counts()

print(domicilio_counts[domicilio_counts<100], residenza_counts[residenza_counts<100])

frequenze_PU = df_new[df_new['Provincia_Domicilio'] == 'PU'].shape[0]
print("Absolute frequencies of 'PU':", frequenze_PU)

frequenze_FC = df_new[df_new['Provincia_Domicilio'] == 'FC'].shape[0]
print("Absolute frequencies of 'FC':", frequenze_FC)

Provincia_Domicilio
NU    98
IS    96
VS    90
OG    29
FO    17
PS     8
Name: count, dtype: int64 Provincia_Residenza
VS    95
OG    41
FO     1
Name: count, dtype: int64
Absolute frequencies of 'PU': 1054
Absolute frequencies of 'FC': 968


In [5]:
df_new['Provincia_Residenza'] = df_new['Provincia_Residenza'].replace('FO', 'FC')
df_new['Provincia_Domicilio'] = df_new['Provincia_Domicilio'].replace('FO', 'FC')

df_new['Provincia_Residenza'] = df_new['Provincia_Residenza'].replace('PS', 'PU')
df_new['Provincia_Domicilio'] = df_new['Provincia_Domicilio'].replace('PS', 'PU')

domicilio_counts = df_new['Provincia_Domicilio'].value_counts()
residenza_counts = df_new['Provincia_Residenza'].value_counts()

print(domicilio_counts[domicilio_counts<100], residenza_counts[residenza_counts<100])

Provincia_Domicilio
NU    98
IS    96
VS    90
OG    29
Name: count, dtype: int64 Provincia_Residenza
VS    95
OG    41
Name: count, dtype: int64


###### Insted of changing all categorical features in dummies, there are some ordinal categorical features that we can encode.

In [6]:
mapping = {
    'CAUTO': 1,
    'PRUDENTE': 2,
    'BILANCIATO': 3,
    'DINAMICO':4,
    'ND':0
}
df_new['Profilo_MIFID'] = df_new['Profilo_MIFID'].replace(mapping)

mapping2 = {
    'Bassissimo (<1200)': 1,
    'Basso (tra 1200 e 1500)': 2,
    'Medio (tra 1500 e 1800)': 3,
    'Alto (tra 1800 e 2500)':4,
    'Altissimo (>2500)':5
}

df_new['Imp_Reddito'] = df_new['Imp_Reddito'].replace(mapping2)

#### Numerical features

In [7]:
df_new.describe(include=['int64','float64'])

,Id_Cliente,Imp_Valore_del_Cliente,Eta,Anno_Apertura_primo_Conto,Imp_Reddito,Profilo_MIFID,Imp_Canone_Pricing_cc,Imp_Bonifici_vs_Competitors,Imp_Bonifici_vs_No_Competitors,Num_Utenze_in_essere,...,Num_attività_trading,Var_Numero_Utenze,Iscrizione_programma_Loyalty,Num_Punti_Programma_Loyalty,Imp_Liquidità_Attuale,Imp_Gestito_attuale,Imp_Amministrato_attuale,Imp_Liquidità_Attuale_6m,Imp_Gestito_attuale_6m,Imp_Amministrato_attuale_6m
count,175331.000000,175329.000000,175330.000000,175331.000000,72783.000000,152455.000000,162724.000000,41620.000000,93528.000000,97637.000000,...,40380.000000,101509.000000,175331.000000,59106.000000,1.753260e+05,5.164900e+04,8.692800e+04,1.753210e+05,5.156600e+04,8.632800e+04
mean,273525.021548,638.247857,46.014407,2004.819422,3.200514,2.554360,0.909484,3132.650691,3287.028303,4.472126,...,15.025978,0.190525,0.337111,1035.779447,1.513416e+04,6.925936e+04,7.726310e+04,1.536550e+04,6.713867e+04,7.378483e+04
std,157999.935555,2652.963429,13.157246,3.281109,1.367305,1.448426,1.829219,13286.878472,14455.623836,3.654871,...,95.042968,1.081378,0.472724,3761.446648,4.506248e+04,2.013812e+05,2.538956e+05,4.504667e+04,1.953801e+05,2.475000e+05
min,1.000000,0.000000,3.000000,1994.000000,1.000000,0.000000,0.000000,0.010000,0.270000,1.000000,...,1.000000,-22.000000,0.000000,-25.000000,-1.996979e+06,0.000000e+00,0.000000e+00,-1.995983e+06,0.000000e+00,0.000000e+00
25%,136846.500000,47.070000,37.000000,2002.000000,2.000000,2.000000,0.000000,160.000000,350.000000,2.000000,...,1.000000,0.000000,0.000000,150.000000,6.251575e+02,7.868200e+03,5.096505e+03,6.291500e+02,7.743670e+03,5.025382e+03
50%,273044.000000,97.270000,43.000000,2005.000000,3.000000,3.000000,0.000000,480.000000,800.000000,3.000000,...,2.000000,0.000000,0.000000,490.000000,3.849440e+03,2.317491e+04,2.105812e+04,3.808250e+03,2.252587e+04,2.019604e+04
75%,410619.500000,370.440000,54.000000,2008.000000,4.000000,4.000000,0.475000,1484.732500,2018.010833,6.000000,...,6.000000,0.000000,1.000000,1070.000000,1.416257e+04,6.453575e+04,6.889509e+04,1.405026e+04,6.287694e+04,6.541610e+04
max,547434.000000,260807.280000,132.000000,2010.000000,5.000000,4.000000,5.950000,510000.000000,1000000.000000,43.000000,...,7816.000000,22.000000,1.000000,587820.000000,4.702548e+06,1.319741e+07,2.444732e+07,4.418061e+06,1.298017e+07,2.382552e+07


#### Dropping of some features


In [9]:
df_new = df_new.drop(['Imp_Gestito_attuale_6m', 'Imp_Liquidità_Attuale_6m', 'Imp_Amministrato_attuale_6m', 'Id_Cliente'], axis =1)
df_new.head()

,Imp_Valore_del_Cliente,Flag_Apertura_Conto_Online,Flag_Possesso_piu_Conti,Eta,Provincia_Domicilio,Provincia_Residenza,Anno_Apertura_primo_Conto,Professione,Imp_Reddito,Tipologia_Cliente,...,Flag_Contatto_Call_Center,Flag_variazione_Accredito_Stipendio,Num_Punti_Programma_Loyalty,Imp_Liquidità_Attuale,Imp_Gestito_attuale,Imp_Amministrato_attuale,Flag_Trasferimento_Titoli_Out,Flag_Rifiuto_Carte,Flag_Rifiuto_Prestiti,Flag_Disattivazione_RID
0,270.60,0,0,62.0,PG,PG,2009.0,Impiegato,3.0,Investor,...,0.0,0.0,NaN,9194.79,36100.96,71113.80,0.0,0.0,0.0,0.0
1,43.06,0,0,32.0,BO,BO,2010.0,Libero professionista/Titolare impresa,4.0,Banker,...,0.0,0.0,435.0,16041.01,NaN,NaN,0.0,0.0,0.0,0.0
2,1445.58,0,0,68.0,LC,LC,2004.0,Libero professionista/Titolare impresa,NaN,Investor,...,0.0,NaN,NaN,17708.33,107818.19,107812.62,0.0,0.0,0.0,0.0
3,912.72,0,0,81.0,MN,MN,2007.0,Altro/Nessuno,NaN,Investor,...,0.0,NaN,NaN,231.38,54271.22,70986.43,0.0,0.0,0.0,0.0
4,36.78,1,1,37.0,MI,MI,2009.0,Impiegato,NaN,Banker,...,1.0,NaN,75.0,222.48,NaN,NaN,0.0,0.0,0.0,1.0


#### Creation of the new dataset with the changes applied

In [10]:
df_new.to_csv("Churn_clean_competition.csv", index=False)